In [312]:
# The idea was train multiple model group by group's secuence after all we simple average them over unseen data
# other ensemble models could by applied
# we use autoencode and raw data to train final model and apply special netrualization feature to final preds
# this model was constructer to compete in "numerai" competition the final results on live data are NOT good...
# but I saved this model in future we can improve

In [1]:
TAG="RANK_SAVED_BEST_VAL_GROUP-{}"
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold

from tqdm import tqdm
from random import choices
# perform GMM clustering over dataset
from sklearn import mixture


from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

import kerastuner as kt

physical_devices = tf.config.list_physical_devices('GPU')
try:
          tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
          # Invalid device or cannot modify virtual devices once initialized.
    print("Err")
    pass

In [2]:
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [270]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import sys
if not sys.warnoptions:
    import warnings
    
    warnings.simplefilter("ignore")

In [3]:
# NN
import random
import os
def seed_everything(seed : int):    
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

seed_everything(1234)    

In [4]:
# download the latest training dataset (takes around 30s)
# training_data = pd.read_csv("latest_numerai_training_data.csv.xz")
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
training_data.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,...,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75,0.50
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,...,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00,0.25
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,...,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75,0.25
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,...,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00,0.25
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,...,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75,0.75


In [5]:
# download the latest tournament dataset (takes around 30s)
# tournament_data = pd.read_csv("latest_numerai_tournament_data.csv.xz")
tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
tournament_data.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.0,0.75,0.5,...,0.75,0.75,1.00,0.75,0.50,0.5,1.0,0.00,0.00,0.25
1,n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.5,0.00,0.0,...,0.50,0.50,0.75,1.00,0.75,0.5,0.5,0.50,0.50,0.50
2,n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.0,1.00,1.0,...,0.00,0.00,0.50,0.25,0.00,0.0,0.5,0.50,0.00,1.00
3,n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.0,0.00,0.0,...,0.00,0.00,0.00,0.25,0.00,0.0,0.0,0.25,0.25,0.50
4,n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.0,0.50,0.5,...,0.50,0.75,0.00,0.00,0.75,1.0,0.0,0.25,1.00,0.75


In [6]:
TARGET_NAME="target"

feature_cols = tournament_data.columns[tournament_data.columns.str.startswith('feature')]
feats_and_target = feature_cols.to_list().copy()
feats_and_target.append(TARGET_NAME)


In [7]:
validation = tournament_data[tournament_data["data_type"]=="validation"].copy()
validation

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.00,0.75,0.50,...,0.75,0.75,1.00,0.75,0.50,0.50,1.00,0.00,0.00,0.25
1,n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.50,0.00,0.00,...,0.50,0.50,0.75,1.00,0.75,0.50,0.50,0.50,0.50,0.50
2,n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.00,1.00,1.00,...,0.00,0.00,0.50,0.25,0.00,0.00,0.50,0.50,0.00,1.00
3,n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.00,0.00,0.00,...,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.50
4,n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.00,0.50,0.50,...,0.50,0.75,0.00,0.00,0.75,1.00,0.00,0.25,1.00,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545360,nffb61f786fe2a96,era212,validation,0.00,0.50,0.75,0.00,0.50,0.50,0.50,...,1.00,1.00,1.00,1.00,1.00,0.75,0.75,0.75,1.00,0.50
1545361,nffbe101615ad597,era212,validation,0.75,0.75,0.75,0.75,0.25,0.25,0.25,...,0.75,0.75,0.25,0.25,0.75,0.75,0.25,0.50,0.75,0.25
1545362,nffc1dc801a3318a,era212,validation,0.25,1.00,0.50,0.00,0.00,0.25,0.50,...,0.25,0.50,0.00,0.00,0.25,0.25,0.00,0.75,0.75,0.50
1545363,nffc376c3127112d,era212,validation,0.00,0.75,0.75,0.00,0.75,0.50,0.75,...,0.50,0.50,0.75,0.75,0.25,0.25,0.75,0.50,0.50,1.00


In [178]:
validation = validation.reset_index().reindex().drop('index', axis=1)

In [8]:
allData = pd.concat((training_data, tournament_data))
allData

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.50,0.25,0.25,...,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75,0.50
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.50,0.00,0.00,...,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00,0.25
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.00,0.75,0.75,...,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75,0.25
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.50,0.25,0.25,...,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00,0.25
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.00,0.25,0.50,...,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1665893,nffcbaaa118a9c2a,eraX,live,0.50,0.75,0.50,1.00,0.25,0.25,0.25,...,0.50,0.00,0.25,0.25,0.00,0.25,0.25,0.25,0.25,NaN
1665894,nffd1dc6e0f798a6,eraX,live,1.00,0.50,0.75,0.75,1.00,0.75,0.75,...,0.00,0.00,0.00,0.00,0.25,0.75,0.00,0.50,0.00,NaN
1665895,nffdfcf0fb116595,eraX,live,1.00,1.00,1.00,1.00,0.00,0.00,0.00,...,0.00,0.25,0.00,0.00,0.25,0.50,0.00,0.25,0.25,NaN
1665896,nfff187c15c3ede1,eraX,live,0.75,0.50,0.50,0.75,0.25,0.25,0.25,...,0.00,0.00,1.00,1.00,0.25,1.00,1.00,0.00,0.25,NaN


In [117]:
from sklearn.model_selection import train_test_split
from sklearn import decomposition
import umap
SEED=1234



def generateEncodeModel(num_feats=310):
    """
    yield a simple MLP with metric learning
    
    """


    # adapted from https://github.com/ghmagazine/kagglebook/blob/master/ch06/ch06-03-hopt_nn.py
    params = {
        'input_dropout': 0.0,
        'hidden_layers': 4,
        'hidden_units': 512,
        'hidden_activation': 'relu',
        'dropout': 0.1,
        'lr': 1e-3,
        'batch_size': 10000,
        'epochs': 192
    }
    # NN model architecture
    n_neuron = params['hidden_units']

    inputs = tf.keras.layers.Input(shape=(num_feats, ))
    x = tf.keras.layers.Dense(n_neuron, activation=params['hidden_activation'])(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(params['dropout'])(x)

    # stack hidden layers
    for i in np.arange(params['hidden_layers'] - 1):
        x = tf.keras.layers.Dense(n_neuron // (2 * (i+1)), activation=params['hidden_activation'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(params['dropout'])(x)
        
    x = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1), name='l2_norm')(x) # L2 normalize embeddings!
    out = tf.keras.layers.Dense(num_feats,)(x)
    out2 = tf.keras.layers.Dense(1,name='l_neut')(x)
   
    # output
    model = tf.keras.models.Model(inputs=inputs, outputs=[out, out2])

    # compile
    loss = tf.keras.losses.MeanSquaredError()
    opt = tf.keras.optimizers.Adam(lr=params['lr'])
    model.compile(loss=loss, optimizer=opt)
    
    return model

# extract embedding features

def extract_emb(best_model, x_train, layer_name='l2_norm', method='umap'):
    
    hidden_model = tf.keras.models.Model(inputs=best_model.input,
                         outputs=best_model.get_layer(layer_name).output)

    # input train, get embedded outputs
    train_dist = hidden_model.predict(x_train)
    
    return train_dist

model = generateEncodeModel(310)
model.summary()

Model: "model_43"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_30 (InputLayer)           [(None, 310)]        0                                            
__________________________________________________________________________________________________
dense_139 (Dense)               (None, 512)          159232      input_30[0][0]                   
__________________________________________________________________________________________________
batch_normalization_105 (BatchN (None, 512)          2048        dense_139[0][0]                  
__________________________________________________________________________________________________
dropout_105 (Dropout)           (None, 512)          0           batch_normalization_105[0][0]    
___________________________________________________________________________________________

In [23]:
Ranker.AE_MODEL_PATH

'./ae_model'

In [256]:
from sklearn.linear_model import LinearRegression

def neutralize_series(Y, X, data):
    """
    neutralize pandas series (originally from the Numerai Tournament)
    """
    lm = LinearRegression()
    lm.fit(data[[Y]], data[X])
    neutralized_feature = data[X] - lm.predict(data[[Y]])
    
    return neutralized_feature, lm

def neutralize_group(train, target='target', feature_cols=[]):
    allnts = []
    for f in feature_cols:
    # neut
        neut, lm = neutralize_series(target, f, train)      
        # assign
        train[f] = neut.values
        allnts.append(lm)
    return train, allnts

def neutralize(series,by, proportion):
    scores = series.values.reshape(-1, 1)
    exposures = by.values.reshape(-1, 1)
    exposures = np.hstack((exposures, np.array([np.mean(series)] * len(exposures)).reshape(-1, 1)))
    correction = proportion * (exposures.dot(np.linalg.lstsq(exposures, scores)[0]))
    corrected_scores = scores - correction
    neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
    return neutralized

def spearmanr(target, pred):
    return np.corrcoef(
        target,
        pred.rank(pct=True, method="first")
    )[0, 1]
def ar1(x):
    return np.corrcoef(x[:-1], x[1:])[0,1]


def finalneutralize(series,by, proportion):
    scores = series.values.reshape(-1, 1)
    exposures = by.values.reshape(-1, 1)
    exposures = np.hstack((exposures, np.array([np.mean(series)] * len(exposures)).reshape(-1, 1)))
    correction = proportion * (exposures.dot(np.linalg.lstsq(exposures, scores)[0]))
    corrected_scores = scores - correction
    neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
    return neutralized
    

In [310]:


class Ranker():
    
    AE_MODEL_PATH='./ae_model'
    def __init__(self, 
                 fullData=None, 
                 groupbyField='era', 
                 train_dataset=None, 
                 validation_dataset=None, 
                 numNeutralizedFeats=10,
                 feature_cols=None,
                 target_col=None,                 
                 encode_model_path=None,
                 neutralizedTrain=None
                ):
        
        self.fullData = fullData
        self.groupbyField=groupbyField
        self.train_dataset = train_dataset
        self.validation_dataset = validation_dataset
        self.numNeutralizedFeats = numNeutralizedFeats
        self.feature_cols = feature_cols
        self.target_col = target_col
        self.feats_and_target = self.feature_cols.to_list().copy()
        self.feats_and_target.append(self.target_col)        
        self.encode_model_path = encode_model_path
        self.neutralizedTrain=neutralizedTrain
        
        self.neutralizeTrain()
        self.reduceDimension()

        
    def reduceDimension(self):
        """
        We need reduce the original dataset dimension in order the reduce the dimensionality 
        and perform the operations faster
        By removing some high correlated feature by netrualize them again other feats
        we can improve the generalitzacion on unseen data
        Experiment and determine the number of gold features
        Final and trainable dataset should be = numNetualizedFeats*(encode_dim-numNeutralizedFeatures)
        """
        
        if self.encode_model_path:
            self.encode_model = generateEncodeModel(len(self.feature_cols))
            self.encode_model.load_weights(self.encode_model_path)
        else:
            self.encode_model = generateEncodeModel(len(self.feature_cols))
            
            
            X_train, X_test, y_train, y_test, y_target_train, y_target_test = train_test_split(self.train_dataset[self.feature_cols].values, 
                                                                self.neutralizedTrain[self.feature_cols].values, 
                                                                self.train_dataset[self.target_col].values,                                                                                                
                                                                test_size=0.33, 
                                                                random_state=42)
            
            train_set = {'X': X_train, 'y':[y_train,y_target_train]}
            valid_set = {'X': X_test, 'y': [y_test, y_target_test]}
            early_stop = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True, monitor='val_loss')
            reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1, mode='min')
            model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=Ranker.AE_MODEL_PATH, 
                                    save_weights_only=True, verbose=0, monitor='val_loss', save_best_only=True)

            # fit            
            self.ae_history = self.encode_model.fit(train_set['X'], train_set['y'], 
                                   callbacks=[reduce_lr, model_checkpoint_callback, early_stop], 
                                   verbose=2, epochs=150, batch_size=10000,
                                   validation_data=(valid_set['X'], valid_set['y'])) 
            
            self.encode_model.save_weights(Ranker.AE_MODEL_PATH)
            
        pass
        
    def prepareTrainableDataSet(self):
        # 1.- Autoencode for train and autoencode port validation
        self.train_encode = extract_emb(self.encode_model, self.train_dataset[self.feature_cols].values)
        self.val_encode = extract_emb(self.encode_model, self.validation_dataset[self.feature_cols].values)
        
        
        self.features_encoded = [x for x in range(self.train_encode.shape[1])]
        
        self.feats_and_targetneu = self.feature_cols.to_list().copy()
        self.feats_and_targetneu.append('pred_neut')     
        
        # for train dataset
        by = extract_emb(self.encode_model, self.train_dataset[self.feature_cols].values, layer_name='l_neut')
        
        self.train_dataset['pred_neut'] = by        

        
        allFrames = []
        for group in self.train_dataset[self.groupbyField].unique():

            train, _ = neutralize_group(self.train_dataset[self.train_dataset[self.groupbyField] == group][self.feats_and_targetneu].copy(), target='pred_neut', feature_cols=self.feature_cols)
            allFrames.append(train)
                    
        neutData = pd.concat(allFrames, axis=0) 
        
        self.train_encode=pd.DataFrame(self.train_encode)
        
        self.train_encode['group'] = self.train_dataset[self.groupbyField].values
        self.train_encode['target'] = self.train_dataset[self.target_col].values
        
        # used for train raw neutrailzed data and encoders layer l2_norm
        self.train_encode = pd.concat((self.train_encode, neutData), axis=1)
        
        
        # for validation dataset
        by = extract_emb(self.encode_model, self.validation_dataset[self.feature_cols].values, layer_name='l_neut')
        self.validation_dataset['pred_neut'] = by        
        
        allFrames = []
        for group in self.validation_dataset[self.groupbyField].unique():

            train, _ = neutralize_group(self.validation_dataset[self.validation_dataset[self.groupbyField] == group][self.feats_and_targetneu].copy(), target='pred_neut', feature_cols=self.feature_cols)
            allFrames.append(train)
                    
        neutData = pd.concat(allFrames, axis=0) 
        
        
        self.val_encode=pd.DataFrame(self.val_encode)
        
        self.val_encode['group'] = self.validation_dataset[self.groupbyField].values
        self.val_encode['target'] = self.validation_dataset[self.target_col].values
        
        # used for val raw neutrailzed data and encoders layer l2_norm
        self.val_encode = pd.concat((self.val_encode, neutData), axis=1)
        print("DataSet Ready")
        
    def neutralizeTrain(self):
        
        if not isinstance(self.neutralizedTrain,pd.DataFrame):
            allFrames = []
            allNts = []
            for group in self.train_dataset[self.groupbyField].unique():

                train, _ = neutralize_group(self.train_dataset[self.train_dataset[self.groupbyField] == group][self.feats_and_target].copy(), target=self.target_col, feature_cols=self.feature_cols)
                allFrames.append(train)
                allNts.append(_)
    
            self.neutralizedTrain = pd.concat(allFrames, axis=0)    

    def perdict_int(self, data):
        data = data.copy()
        data = data.reset_index().reindex().drop('index', axis=1)    
        encode_data = extract_emb(self.encode_model, data[self.feature_cols].values)    
        by = extract_emb(self.encode_model, data[self.feature_cols].values, layer_name='l_neut')

        data['pred_neut'] = by        
        dataNeut, _ = neutralize_group(data[self.feats_and_targetneu].copy(), target='pred_neut', feature_cols=self.feature_cols)        

        data=pd.DataFrame(encode_data)

        # used for val raw neutrailzed data and encoders layer l2_norm
        data = pd.concat((data, dataNeut), axis=1)

        feats =  self.features_encoded + [col for col in self.feature_cols]

        pred = np.mean([finalneutralize(pd.Series(model.predict(data[feats].values)), 
                                        pd.Series(by.reshape(-1)), 0.8).values for model in self.allmodels], 
                        axis=0)

        return pred

    def predict_val(self):
        scores = []

        for group in self.validation_dataset[self.groupbyField].unique():    
            predictions = self.perdict_int(self.validation_dataset[self.validation_dataset[self.groupbyField]==group])
            score = np.corrcoef(self.validation_dataset[self.validation_dataset[self.groupbyField]==group]['target'], 
                                pd.Series(predictions).rank(pct=True, method="first"))[0][1]
            scores.append(score)
            print("Score group {} - correlation {}".format(group, score))

        print("Autocrrelation", ar1(scores))
        print("AVG Score:",np.mean(scores))
        print("Rshape:",np.mean(scores)/np.std(scores))
        return np.mean(scores)/np.std(scores)  

    def train_groups(self):
        self.allmodels = []
        feats =  self.features_encoded + [col for col in self.feature_cols]

        for x in range(len(self.train_encode['group'].unique())-5):
        #for x in range(100):            
            # train_groups = np.random.choice(self.train_encode['group'].unique(), 70)
            train_groups = self.train_encode['group'].unique()[x:x+5]
            print("train groups", train_groups)
            train_X = self.train_encode[self.train_encode['group'].isin(train_groups)][feats].values
            train_y = self.train_encode[self.train_encode['group'].isin(train_groups)]['target'].values
            test_X = self.train_encode[~self.train_encode['group'].isin(train_groups)][feats].values
            test_y = self.train_encode[~self.train_encode['group'].isin(train_groups)]['target'].values

            model = CatBoostRegressor(iterations=1000,    
                            verbose=False,
                            #use_best_model=True,
                            # depth=6,
                            # subsample=0.7,
                            # l2_leaf_reg=7,
                            # random_strength=0.2,
                            # bagging_temperature=0.3,

                            # learning_rate=0.003,
                            #     custom_loss=['Accuracy'],
                            #     eval_metric='Accuracy',   
                            task_type="GPU",  
                           devices='0:1')

            model.fit(train_X, train_y, 
                      # eval_set = (test_X, test_y), 
                      # early_stopping_rounds=200, 
                      plot=False)   
            self.allmodels.append(model)

            if x%5==0 and x>0:
                self.predict_val()    
                
    def load_models(self):
        #TODO
        pass
    def save_models(self):
        #TODO
        pass
    def train(self):
        self.prepareTrainableDataSet()
        self.train_groups()

In [112]:
ranker = Ranker(fullData=allData,
               groupbyField='era', 
               train_dataset=training_data, 
               validation_dataset=validation, 
               numNeutralizedFeats=10,
               feature_cols = feature_cols,
               target_col = TARGET_NAME,               
               encode_model_path=None,
               neutralizedTrain=None
              )    

ranker.train()

Epoch 1/150
34/34 - 2s - loss: 0.2935 - dense_131_loss: 0.1158 - dense_132_loss: 0.1777 - val_loss: 0.2368 - val_dense_131_loss: 0.1238 - val_dense_132_loss: 0.1130
Epoch 2/150
34/34 - 1s - loss: 0.1712 - dense_131_loss: 0.1082 - dense_132_loss: 0.0631 - val_loss: 0.1902 - val_dense_131_loss: 0.1180 - val_dense_132_loss: 0.0723
Epoch 3/150
34/34 - 1s - loss: 0.1551 - dense_131_loss: 0.0980 - dense_132_loss: 0.0571 - val_loss: 0.1834 - val_dense_131_loss: 0.1151 - val_dense_132_loss: 0.0682
Epoch 4/150
34/34 - 1s - loss: 0.1473 - dense_131_loss: 0.0914 - dense_132_loss: 0.0559 - val_loss: 0.1741 - val_dense_131_loss: 0.1052 - val_dense_132_loss: 0.0689
Epoch 5/150
34/34 - 1s - loss: 0.1410 - dense_131_loss: 0.0865 - dense_132_loss: 0.0544 - val_loss: 0.1645 - val_dense_131_loss: 0.0950 - val_dense_132_loss: 0.0695
Epoch 6/150
34/34 - 1s - loss: 0.1363 - dense_131_loss: 0.0828 - dense_132_loss: 0.0535 - val_loss: 0.1555 - val_dense_131_loss: 0.0881 - val_dense_132_loss: 0.0674
Epoch 7/15

Epoch 51/150
34/34 - 1s - loss: 0.0922 - dense_131_loss: 0.0427 - dense_132_loss: 0.0496 - val_loss: 0.0829 - val_dense_131_loss: 0.0306 - val_dense_132_loss: 0.0523
Epoch 52/150
34/34 - 1s - loss: 0.0921 - dense_131_loss: 0.0425 - dense_132_loss: 0.0496 - val_loss: 0.0828 - val_dense_131_loss: 0.0304 - val_dense_132_loss: 0.0524
Epoch 53/150
34/34 - 1s - loss: 0.0920 - dense_131_loss: 0.0424 - dense_132_loss: 0.0496 - val_loss: 0.0826 - val_dense_131_loss: 0.0304 - val_dense_132_loss: 0.0522
Epoch 54/150
34/34 - 1s - loss: 0.0917 - dense_131_loss: 0.0423 - dense_132_loss: 0.0495 - val_loss: 0.0827 - val_dense_131_loss: 0.0301 - val_dense_132_loss: 0.0526
Epoch 55/150
34/34 - 1s - loss: 0.0916 - dense_131_loss: 0.0421 - dense_132_loss: 0.0494 - val_loss: 0.0823 - val_dense_131_loss: 0.0299 - val_dense_132_loss: 0.0524
Epoch 56/150
34/34 - 1s - loss: 0.0914 - dense_131_loss: 0.0420 - dense_132_loss: 0.0494 - val_loss: 0.0817 - val_dense_131_loss: 0.0298 - val_dense_132_loss: 0.0519
Epoc

In [113]:
copyNeut = ranker.neutralizedTrain.copy()

In [311]:
ranker = Ranker(fullData=allData,
                groupbyField='era', 
                train_dataset=training_data, 
                validation_dataset=validation, 
                numNeutralizedFeats=10,
                feature_cols = feature_cols,
                target_col = TARGET_NAME,                
                encode_model_path=Ranker.AE_MODEL_PATH,
                neutralizedTrain=copyNeut
               )    

ranker.train()

DataSet Ready
train groups ['era1' 'era2' 'era3' 'era4' 'era5']


train groups ['era2' 'era3' 'era4' 'era5' 'era6']


train groups ['era3' 'era4' 'era5' 'era6' 'era7']


train groups ['era4' 'era5' 'era6' 'era7' 'era8']


train groups ['era5' 'era6' 'era7' 'era8' 'era9']


...

train groups ['era111' 'era112' 'era113' 'era114' 'era115']


Score group era121 - correlation 0.012375645939492638
Score group era122 - correlation 0.012234524888617928
Score group era123 - correlation 0.04514383760908958
Score group era124 - correlation 0.06332742309734196
Score group era125 - correlation 0.009734383373200877
Score group era126 - correlation 0.019546588470218566
Score group era127 - correlation 0.0020855067683530266
Score group era128 - correlation 0.03515502064133281
Score group era129 - correlation -0.008286019549452484
Score group era130 - correlation 0.049024154587592704
Score group era131 - correlation 0.016424027720438203
Score group era132 - correlation 0.04461796261942174
Score group era197 - correlation 0.038730329501925724
Score group era198 - correlation -0.001253007843685644
Score group era199 - correlation -0.038285860392194886
Score group era200 - correlation 0.01415302483648824
Score group era201 - correlation 0.03527949179497769
Score group era202 - correlation 0.049793328701356575
Score group era203 - correlati

train groups ['era113' 'era114' 'era115' 'era116' 'era117']


train groups ['era114' 'era115' 'era116' 'era117' 'era118']


train groups ['era115' 'era116' 'era117' 'era118' 'era119']


In [307]:
def perdict_int(self, data):
    data = data.copy()
    data = data.reset_index().reindex().drop('index', axis=1)    
    encode_data = extract_emb(self.encode_model, data[self.feature_cols].values)    
    by = extract_emb(self.encode_model, data[self.feature_cols].values, layer_name='l_neut')

    data['pred_neut'] = by        
    dataNeut, _ = neutralize_group(data[self.feats_and_targetneu].copy(), target='pred_neut', feature_cols=self.feature_cols)        

    data=pd.DataFrame(encode_data)

    # used for val raw neutrailzed data and encoders layer l2_norm
    data = pd.concat((data, dataNeut), axis=1)

    feats =  self.features_encoded + [col for col in self.feature_cols]
    
    pred = np.mean([finalneutralize(pd.Series(model.predict(data[feats].values)), 
                                    pd.Series(by.reshape(-1)), 0.8).rank(method="first", pct=True).values for model in self.allmodels], 
                   axis=0)
    # pred = np.mean([pd.Series(model.predict(data[feats].values)).rank(method="first", pct=True).values for model in self.allmodels], 
#                   axis=0)
    return pred
def predict_val(self):
    scores = []

    for group in self.validation_dataset[self.groupbyField].unique():    
        predictions = perdict_int(self, self.validation_dataset[self.validation_dataset[self.groupbyField]==group])
        score = np.corrcoef(self.validation_dataset[self.validation_dataset[self.groupbyField]==group]['target'], 
                            pd.Series(predictions).rank(pct=True, method="first"))[0][1]
        scores.append(score)
        print("Score group {} - correlation {}".format(group, score))

    print("Autocrrelation", ar1(scores))
    print("AVG Score:",np.mean(scores))
    print("Rshape:",np.mean(scores)/np.std(scores))
    return np.mean(scores)/np.std(scores)  
predict_val(ranker)

Score group era121 - correlation -0.01954358027672134
Score group era122 - correlation 0.0044214270913480016
Score group era123 - correlation 0.03217119148974253
Score group era124 - correlation 0.013672961214451124
Score group era125 - correlation -0.0013114874186980704
Score group era126 - correlation -0.012976513268435203
Score group era127 - correlation -0.007619394832216456
Score group era128 - correlation 0.03587332575004587
Score group era129 - correlation 0.015641244246493897
Score group era130 - correlation 0.008069727248534943
Score group era131 - correlation -0.02380810683905939
Score group era132 - correlation 0.04158011966135679
Score group era197 - correlation 0.008866022502827947
Score group era198 - correlation -0.019823426707574466
Score group era199 - correlation -0.04182223305720973
Score group era200 - correlation 0.015567626233614478
Score group era201 - correlation 0.012405146316719077
Score group era202 - correlation 0.019856242707703607
Score group era203 - corr

0.31633168322617483

In [181]:
ranker.prepareTrainableDataSet()

In [213]:
ranker.encode_model

In [227]:
def perdict_int(self, data):
    data = data.copy()
    data = data.reset_index().reindex().drop('index', axis=1)    
    encode_data = extract_emb(self.encode_model, data[self.feature_cols].values)    
    by = extract_emb(self.encode_model, data[self.feature_cols].values, layer_name='l_neut')
    
    data['pred_neut'] = by        
    dataNeut, _ = neutralize_group(data[self.feats_and_targetneu].copy(), target='pred_neut', feature_cols=self.feature_cols)        

    data=pd.DataFrame(encode_data)
        
    # used for val raw neutrailzed data and encoders layer l2_norm
    data = pd.concat((data, dataNeut), axis=1)
    
    feats =  self.features_encoded + [col for col in self.feature_cols]
    
    pred = np.mean([model.predict(data[feats].values) for model in self.allmodels], axis=0)
    
    return pred
    
def predict_val(self):
    scores = []
    
    for group in self.validation_dataset[self.groupbyField].unique():    
        predictions = perdict_int(ranker, self.validation_dataset[self.validation_dataset[self.groupbyField]==group])
        score = np.corrcoef(self.validation_dataset[self.validation_dataset[self.groupbyField]==group]['target'], pd.Series(predictions).rank(method="first"))[0][1]
        scores.append(score)
        print("Score group {} - correlation {}".format(group, score))
        
    print("Autocrrelation", ar1(scores))
    print("AVG Score:",np.mean(scores))
    print("Rshape:",np.mean(scores)/np.std(scores))
    return np.mean(scores)/np.std(scores)

predict_val(ranker)

Score group era121-cor 0.0006133815696893267
Score group era122-cor 0.007338712629087977
Score group era123-cor 0.033459988221705116
Score group era124-cor 0.02015197257977349
Score group era125-cor 0.014886159659765948
Score group era126-cor 0.03357480202876932
Score group era127-cor 0.011800464530482132
Score group era128-cor 0.03750488646270648
Score group era129-cor -0.009492144605805867
Score group era130-cor 0.011964417020186207
Score group era131-cor 0.021059993283445967
Score group era132-cor 0.05463504130078883
Score group era197-cor 0.008766118798058864
Score group era198-cor -0.04066689230358891
Score group era199-cor -0.03364164878451009
Score group era200-cor -0.000162524602958868
Score group era201-cor 0.014697064236858457
Score group era202-cor 0.026154350200594118
Score group era203-cor 0.018528270655674448
Score group era204-cor 0.02805347607860009
Score group era205-cor -0.0017222117385623237
Score group era206-cor 0.017308012033776254
Score group era207-cor 0.0232201

0.5600744937661672

In [208]:

    

def train_groups(self):
    self.allmodels = []
    feats =  self.features_encoded + [col for col in self.feature_cols]
    
    for x in range(len(self.train_encode['group'].unique())-4):
        
        train_groups = self.train_encode['group'].unique()[x:x+4]
        print("train groups", train_groups)
        train_X = self.train_encode[self.train_encode['group'].isin(train_groups)][feats].values
        train_y = self.train_encode[self.train_encode['group'].isin(train_groups)]['target'].values
        test_X = self.train_encode[~self.train_encode['group'].isin(train_groups)][feats].values
        test_y = self.train_encode[~self.train_encode['group'].isin(train_groups)]['target'].values

        model = CatBoostRegressor(iterations=10000,    
                        verbose=False,
                        use_best_model=True,
                        task_type="GPU",
                       devices='0:1')
    
        model.fit(train_X, train_y, eval_set = (test_X, test_y), early_stopping_rounds=200, plot=False)   
        self.allmodels.append(model)
        
        if x%5==0 and x>0:
            self.predict_val()
        
train_groups(ranker)    

train groups ['era1' 'era2' 'era3' 'era4']
train groups ['era2' 'era3' 'era4' 'era5']


KeyboardInterrupt: 

In [190]:
ranker.feature_cols

Index(['feature_intelligence1', 'feature_intelligence2',
       'feature_intelligence3', 'feature_intelligence4',
       'feature_intelligence5', 'feature_intelligence6',
       'feature_intelligence7', 'feature_intelligence8',
       'feature_intelligence9', 'feature_intelligence10',
       ...
       'feature_wisdom37', 'feature_wisdom38', 'feature_wisdom39',
       'feature_wisdom40', 'feature_wisdom41', 'feature_wisdom42',
       'feature_wisdom43', 'feature_wisdom44', 'feature_wisdom45',
       'feature_wisdom46'],
      dtype='object', length=310)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 'feature_intelligence1',
 'feature_intelligence2',
 'feature_intelligence3',
 'feature_intelligence4',
 'feature_intelligence5',
 'feature_intelligence6',
 'feature_intelligence7',
 'feature_intelligence8',
 'feature_intelligence9',
 'feature_intelligence10',
 'feature_intelligence11',
 'feature_intelligence12',
 'feature_charisma1',
 'feature_charisma2',
 'feature_charisma3',
 'feature_charisma4',
 'feature_charisma5',
 'feature_charisma6',
 'feature_charisma7',
 'feature_charisma8',
 'feature_charisma9',
 'feature_charisma10',
 'feature_charisma11',
 'feature_charisma12',
 'f